In [27]:
import urllib.request 
import os
from os import listdir
import pandas as pd
import nltk  
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jjanko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
cwd = os.getcwd()

In [29]:
data_direc = cwd.split('\\')
scrap_direc = os.path.join('\\'.join(data_direc[0:-1]),'data','risk_section')
mapping_direc = os.path.join('\\'.join(data_direc[0:-1]),'data', 'brochure_mapping')
temp_direc = os.path.join('\\'.join(data_direc[0:-1]),'data','temp')
pdf_direc = os.path.join('\\'.join(data_direc[0:-1]),'data','pdf')
txt_direc = os.path.join('\\'.join(data_direc[0:-1]),'data','text_files')
code_direc = os.path.join('\\'.join(data_direc[0:-1]),'code')
data_direc = os.path.join('\\'.join(data_direc[0:-1]),'data')


In [30]:
crd_mapping = pd.read_csv(os.path.join(code_direc, 'crd_mapping.csv'))
print(crd_mapping.head(5))

                      mgrname  mgrno                       name1  year  crd  \
0  A. G. EDWARDS & SONS, INC.  24530  a. g. edwards & sons, inc.  2001    4   
1  A. G. EDWARDS & SONS, INC.  24530  a. g. edwards & sons, inc.  2002    4   
2  A. G. EDWARDS & SONS, INC.  24530  a. g. edwards & sons, inc.  2003    4   
3   A.G. EDWARDS & SONS, INC.  24530   a.g. edwards & sons, inc.  2004    4   
4   A.G. EDWARDS & SONS, INC.  24530   a.g. edwards & sons, inc.  2005    4   

                        name2  
0  a. g. edwards & sons, inc.  
1  a. g. edwards & sons, inc.  
2  a. g. edwards & sons, inc.  
3  a. g. edwards & sons, inc.  
4  a. g. edwards & sons, inc.  


In [31]:
_,_, filenames = next(os.walk(mapping_direc))
mapping_df = pd.DataFrame()
for f in filenames:
    df = pd.read_csv(os.path.join(mapping_direc, f),encoding= 'unicode_escape')
    if mapping_df.empty:
        mapping_df = df.copy()
    else:
        mapping_df = pd.concat([mapping_df, df], axis =0 )

mapping_df.loc[:, 'text'] = ''
print(mapping_df.head(5))

                                      FirmName   SECNumber  CRDNumber  \
0  STEELRIVER INFRASTRUCTURE MANAGEMENT US LLC   801-68144     144012   
1      MARINER WEALTH ADVISORS-CINCINNATI, LLC   801-77253     165759   
2                  MORSE CAPITAL PARTNERS, LLC   801-68819     146383   
3                       SYMMETRIC CAPITAL, LLC  801-112533     160936   
4                                RMB WEST, LLC   801-72566     158138   

   FilingID                                       BrochureName  BrochureID  \
0    890848       STEELRIVER FORM ADV PART 2A FINAL (03-29-19)       96860   
1    895872  RIVERPOINT CAPITAL MANAGEMENT, LLC ADV PART 2A...       86524   
2    893108                                        ADV PART 2A      142277   
3    889089         SYMMETRIC CAPITAL, LLC DISCLOSURE BROCHURE       70199   
4    888819                                 PART 2A MARCH 2015      142278   

   BrochureVersion  DateFiled                   PDFFileName text  
0                4  3/27/

In [32]:
mapping_df.loc[:, 'file_name'] = mapping_df['PDFFileName'].str.replace('.pdf', '')

In [33]:
stopwords = pd.read_csv(os.path.join(data_direc, 'StopWords_Generic.txt'), header = None)

In [34]:
stop = stopwords[stopwords.columns[0]].values.tolist()
s = []
for word in stop:
    s.append(word.lower())

In [35]:
def tokenize_file(data, stop):
    tokens = {}
    words = data.split(' ')
    for w in words:
        w = ''.join(ch for ch in w if ch.isalnum())
        w = ''.join(ch for ch in w if ~ch.isdigit())
        if w not in tokens.keys():
            if w not in stop and len(w) > 1:
                tokens[w] = 1
        else:
            tokens[w] = tokens[w] + 1
    tokens = pd.DataFrame.from_dict(tokens, orient = 'index').T
    return tokens

In [36]:

mapping_df['file_name'] = mapping_df['file_name'].apply(lambda x: '_'.join(x.split('_')[0:-1]))

In [37]:
mapping_df

,FirmName,SECNumber,CRDNumber,FilingID,BrochureName,BrochureID,BrochureVersion,DateFiled,PDFFileName,text,file_name
0,STEELRIVER INFRASTRUCTURE MANAGEMENT US LLC,801-68144,144012,890848,STEELRIVER FORM ADV PART 2A FINAL (03-29-19),96860,4,3/27/2015,144012_96860_4_20150327.pdf,,144012_96860_4
1,"MARINER WEALTH ADVISORS-CINCINNATI, LLC",801-77253,165759,895872,"RIVERPOINT CAPITAL MANAGEMENT, LLC ADV PART 2A...",86524,5,3/27/2015,165759_86524_5_20150327.pdf,,165759_86524_5
2,"MORSE CAPITAL PARTNERS, LLC",801-68819,146383,893108,ADV PART 2A,142277,1,3/27/2015,146383_142277_1_20150327.pdf,,146383_142277_1
3,"SYMMETRIC CAPITAL, LLC",801-112533,160936,889089,"SYMMETRIC CAPITAL, LLC DISCLOSURE BROCHURE",70199,4,3/27/2015,160936_70199_4_20150327.pdf,,160936_70199_4
4,"RMB WEST, LLC",801-72566,158138,888819,PART 2A MARCH 2015,142278,1,3/27/2015,158138_142278_1_20150327.pdf,,158138_142278_1
...,...,...,...,...,...,...,...,...,...,...,...
1461,KESTREL PARTNERS LLP,801-99337,226573,1370924,KESTREL PARTNERS LLP,197088,6,1/27/2020,226573_197088_6_20200127.pdf,,226573_197088_6
1462,"EAGAN CAPITAL MANAGEMENT, LLC",801-68860,126599,1371840,"EAGAN CAPITAL MANAGEMENT, LLC",324987,1,1/23/2020,126599_324987_1_20200123.pdf,,126599_324987_1
1463,"ALPHA CAPITAL FAMILY OFFICE, LLC",801-118157,307521,1373746,WRAP FEE PROGRAM BROCHURE,325460,1,1/31/2020,307521_325460_1_20200131.pdf,,307521_325460_1
1464,"ALPHA CAPITAL FAMILY OFFICE, LLC",801-118157,307521,1373746,PART 2A BROCHURE,325461,1,1/31/2020,307521_325461_1_20200131.pdf,,307521_325461_1


In [38]:
crd_mapping[crd_mapping['crd'] == '10046']

,mgrname,mgrno,name1,year,crd,name2


In [39]:
newdf = pd.DataFrame()
crd_new_df = pd.DataFrame()

_, _, filenames = next(os.walk(scrap_direc))
for f in filenames:
    year = int(f.split('_')[-1][0:4])
    n = f.replace('.txt', '')
    n = '_'.join(n.split('_')[0:-1])
    with open (os.path.join(scrap_direc, f), "r", encoding = 'utf-8') as myfile:
        data=myfile.readlines()
        data = data[0]
        data = data.lower()
        mapping_df.loc[mapping_df['file_name'] == n, 'text'] = 'found'
        file_token = tokenize_file(data, s)
        try:
            file_token.index = mapping_df.loc[mapping_df['file_name'] == n].index
            add = pd.concat([mapping_df.loc[mapping_df['file_name'] == n, :], file_token], axis = 1)
            add.loc[:,'year'] = year
            if newdf.empty:
                newdf = add.copy()
            else:
                newdf = pd.concat([newdf, add], axis = 0)
        except:
            pass
        try: 
            mapping = crd_mapping[(crd_mapping['crd'] == add['CRDNumber'].values[0]) & (crd_mapping['year'] == add['year'].values[0])]
            file_token.index = mapping.index
            if len(mapping.index) > 0:
                add = pd.concat([mapping, file_token], axis = 1)
                add.loc[:, 'file'] = f.replace('.txt', '.pdf')
                if crd_new_df.empty:
                    crd_new_df = add.copy()
                else:
                    crd_new_df = pd.concat([crd_new_df, add], axis = 0, ignore_index = True)
        except:
            pass
        

In [40]:
print(crd_new_df.head(100))

                           mgrname  mgrno                           name1  \
0            GAGNON SECURITIES LLC  39567           gagnon securities llc   
1   SIGMA INVESTMENT MANAGEMENT CO  13095  sigma investment management co   
2   SIGMA INVESTMENT MANAGEMENT CO  13095  sigma investment management co   
3   SIGMA INVESTMENT MANAGEMENT CO  13095  sigma investment management co   
4   MCRAE CAPITAL MANAGEMENT, INC.  55220  mcrae capital management, inc.   
..                             ...    ...                             ...   
95        ALDEBARAN FINANCIAL, INC  13847        aldebaran financial, inc   
96      DRIEHAUS CAPITAL MGMT, LLC  23700      driehaus capital mgmt, llc   
97      DRIEHAUS CAPITAL MGMT, LLC  23700      driehaus capital mgmt, llc   
98       WESTPORT ASSET MGMT, INC.  92230       westport asset mgmt, inc.   
99    SCHAFER CULLEN CAP MANAGMENT  76558    schafer cullen cap managment   

    year     crd                                  name2  investment  \
0   

In [41]:
newdf.to_csv('sample_panel.csv')
crd_new_df.to_csv('sample_panel_crd.csv')